In [1]:
import json
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
files = {'debt': 'data/Campaign_Finance_Disclosure_Debt_Data_Current_State.csv',
         'receipt':'data/Campaign_Finance_Disclosure_Receipt_Data_Current_State.csv',
         'filer': 'data/Campaign_Finance_Disclosure_Filer_Data_Current_State.csv',
         'expense': 'data/Campaign_Finance_Disclosure_Expense_Data_Current_State.csv',
         'contributions': 'data/Campaign_Finance_Disclosure_Contributions_Data_2017_State.csv',
         'lobbyists': 'data/Lobbying_Disclosure_Data_2017_State.csv'}

# Utilities

In [3]:
def amount_to_float(amount):
    if type(amount) == float and np.isnan(amount):
        return np.nan
    pattern = re.compile(r'\$(.*)')
    g = re.match(pattern, amount)
    if g:
        return float(g.groups()[0])
    else:
        return np.nan

In [4]:
def extract_lat_long(x):
    pattern = re.compile(r'\((.*), (.*)\)')
    lat, long = np.nan, np.nan
    if type(x) == float and np.isnan(x):
        return lat, long
    for y in x.split('\n'):
        g = re.match(pattern, y)
        if g:
            lat, long = g.groups()
    return float(lat), float(long)


def get_lat_long(x, string=None):
    lat, long = extract_lat_long(x[string + ' Location 1'])
    if np.isnan(lat):
        lat, long = extract_lat_long(x[string + ' Location 2'])
    return '{} {}'.format(lat, long)


def get_df_lat_long(df, string):
    new_pd = df.apply(get_lat_long, axis=1, string=string).str.split(expand=True).astype(float)
    df[['lat', 'long']] = new_pd.copy()
    return df

# Debt

In [5]:
debt = pd.read_csv(files['debt'])
debt.fillna('', inplace=True)
debt['Debt Accrual Date'] = pd.to_datetime(debt['Debt Accrual Date'].astype(str),
                                           format='%Y%m%d.0',
                                           errors='coerce')

FileNotFoundError: File b'data/Campaign_Finance_Disclosure_Debt_Data_Current_State.csv' does not exist

In [ ]:
debt['amount'] = debt['Debt Amount'].apply(amount_to_float)
debt['filer_id'] = debt['Filer Identification Number']
debt = get_df_lat_long(debt, 'Debt Reporting')
debt['address'] = debt.loc[:, 'Debt Reporting Address 1': 'Debt Reporting Zip Code'].apply(' '.join, axis=1)

In [ ]:
debt.head()

In [ ]:
debt.to_pickle('data/debt.pkl')

# Receipt

In [ ]:
receipt = pd.read_csv(files['receipt'])
receipt.fillna('', inplace=True)
receipt['Receipt Date'] = pd.to_datetime(receipt['Receipt Date'].astype(str), format='%Y%m%d.0', errors='coerce')

In [ ]:
receipt['amount'] = receipt['Receipt Amount'].apply(amount_to_float)
receipt['filer_id'] = receipt['Filer Identification Number']
receipt = get_df_lat_long(receipt, 'Receipt')
receipt['address'] = receipt.loc[:, 'Receipt Address 1': 'Receipt Zip Code'].apply(' '.join, axis=1)

In [ ]:
receipt.head()

In [ ]:
receipt.to_pickle('data/receipt.pkl')

# Filer

In [ ]:
filer = pd.read_csv(files['filer'], dtype={'Phone Number': str})
filer.fillna('', inplace=True)

In [ ]:
FILER_TYPE = {1.0: 'candidate',
              2.0: 'committee', 
              3.0: 'lobbyist'}

In [ ]:
filer['type'] = filer['Filer Type'].apply(lambda x: FILER_TYPE.get(x, 'unknown')).astype('category')
filer['filer_id'] = filer['Filer Identification Number']
filer = get_df_lat_long(filer, 'Filer')
filer['address'] = filer.loc[:, 'Filer Address 1': 'Filer Zip Code'].apply(' '.join, axis=1)

In [ ]:
filer.info()

In [ ]:
filer.head()

In [ ]:
filer.groupby('Office').count()

In [ ]:
filer.groupby(['filer_id', 'Filer Name']).count().sort_index(level='filer_id')

In [ ]:
filer.to_pickle('data/filer.pkl')

# Expense

In [ ]:
expense = pd.read_csv(files['expense'])
expense.fillna('', inplace=True)
expense['Expense Date'] = pd.to_datetime(expense['Expense Date'].astype(str), format='%Y%m%d.0', errors='coerce')

In [ ]:
expense['amount'] = expense['Expense Amount'].apply(amount_to_float)
expense['filer_id'] = expense['Filer Identification Number']
expense = get_df_lat_long(expense, 'Expense')
expense['address'] = expense.loc[:, 'Expense Address 1': 'Expense Zip Code'].apply(' '.join, axis=1)

In [ ]:
expense.head()

In [ ]:
expense.to_pickle('data/expense.pkl')

# Contributions

In [ ]:
contrib = pd.read_csv(files['contributions'], dtype={'Employer Zip Code': str})
contrib.fillna('', inplace=True)
contrib['Contribution Date'] = pd.to_datetime(contrib['Contribution Date'].astype(str),
                                              format='%Y%m%d.0', errors='coerce')

In [ ]:
contrib['amount'] = contrib['Contribution Amount'].apply(amount_to_float)
contrib['filer_id'] = contrib['Filer Identification Number']
contrib = get_df_lat_long(contrib, 'Contributor')
contrib['address'] = contrib.loc[:, 'Contributor Address 1': 'Contributor Zip Code'].apply(' '.join, axis=1)

In [ ]:
contrib.head()

In [ ]:
contrib.to_pickle('data/contrib.pkl')

# Lobbyists

In [ ]:
lobby = pd.read_csv(files['lobbyists'])

In [ ]:
lobby.head()

# Example queries

In [ ]:
contrib.groupby(['Filer Identification Number',
                 'Election Year',
                 'Election Cycle']
               ).sum().sort_values('amount', ascending=False).head()

# Use case examples

Focus on the `debt`, `contrib`, `expense`, `filer`, `receipt` dataframes

## Local activity

Find nearby activity, unfettered by time.

In [ ]:
from geopy.distance import great_circle

In [ ]:
import geocoder
g = geocoder.osm('Ebensburg, PA')
address = (g.osm['y'], g.osm['x'])

In [ ]:
address

In [ ]:
def get_dist(x, address=None):
    lat, long = x['lat'], x['long']
    return great_circle(address, (lat, long)).miles
    
def get_distances(address, df):
    return df.apply(get_dist, address=address, axis=1)

In [ ]:
debt[get_distances(address, debt) < 5]

In [ ]:
contrib[get_distances(address, contrib) < 5]

In [ ]:
expense[get_distances(address, expense) < 5]

## Exporting geojson

In [ ]:
from geojson import Feature, FeatureCollection, Point

In [ ]:
def to_geojson(df):
    for i, x in df.iterrows():
        geometry = None
        if not np.isnan(x.lat):
            geometry = Point((x.lat, x.long))
        properties = json.loads(x.to_json())
        yield Feature(geometry=geometry,
                      properties=properties)
    return

In [ ]:
fc = FeatureCollection([y for y in to_geojson(debt)])

## Timeseries visualizations

In [ ]:
ts = contrib[['Contribution Date', 'amount', 'filer_id']].copy()
ts.sort_values(['Contribution Date', 'filer_id'], inplace=True)

In [ ]:
# TODO: be cautious about NaT values

def reshape(df, index=None):
    df = df.set_index(index)
    return df.iloc[:, :-1].fillna(0)

ts_df = ts.groupby('filer_id').apply(reshape, index='Contribution Date')

In [ ]:
ts_df.groupby('filer_id').count().sort_values('amount', ascending=False).head()

In [ ]:
ts_df.xs('8600230').cumsum().plot()

In [ ]:
expense[expense.filer_id=='8200581']

In [ ]:
filer[filer.filer_id.isin(['8600230', '8200581', '7900477', '2002281', '7900366'])]

### `contrib` histogram

In [ ]:
contrib.amount.describe()

In [ ]:
contrib.sort_values('amount', ascending=False).head()

In [ ]:
contrib[contrib.amount > 5000].shape

In [ ]:
contrib[contrib.amount==contrib.amount.max()]

In [ ]:
filer[filer.filer_id == '2005249']

In [ ]:
expense[expense.filer_id == '2005249']

# Heatmaps

In [ ]:
contrib.groupby('filer_id').count().sort_values('amount', ascending=False).head()

In [ ]:
contrib.groupby(['filer_id', 'Contributor']).count().sort_values('amount', ascending=False).head()

In [ ]:
contrib.groupby('Contributor Zip Code').sum()[['amount']].sort_values('amount', ascending=False).head()

In [ ]:
contrib.groupby('filer_id').mean().sort_values('amount', ascending=False).head()

# Play with fuzzywuzzy